     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.4 MB/s eta 0:00:00


In [93]:
import pandas as pd
import time
import datetime as dt
from datetime import datetime
import numpy as np
import pytz
import yfinance as yf

##Fonk

### Fvg

In [ ]:
##FVG
def detect_fvg(df):
    """
    FVG'leri tespit eder ve retest bekler.
    İşlem noktalarını belirleyip stop ve hedefleri hesaplar.
    """
    df = df.copy()

    # Bullish FVG: 1. mumun high'ı, 3. mumun low'undan küçük olmalı
    df['bullish_fvg'] = ((df['High'].shift(3) < df['Low'].shift(1)) &
                      (df['hour_ny'].isin([3, 10, 14]))).astype(int)


    # Bearish FVG: 1. mumun low'u, 3. mumun high'ından büyük olmalı
    df['bearish_fvg'] = ((df['Low'].shift(3) > df['High'].shift(1)).astype(int) &
                        (df['hour_ny'].isin([3, 10, 14]))).astype(int)

    # Retest kontrolü: FVG bölgesine fiyat tekrar dokundu mu?
    df['bullish_fvg_retest'] = ((df['Low'] <= df['High'].shift(3)) &
                            (df['bullish_fvg'].shift(1) == 1) &
                            (df['hour_ny'].isin([3, 10, 14]))).astype(int)
    df['bearish_fvg_retest'] = ((df['High'] >= df['Low'].shift(2)) &
                                (df['bearish_fvg'].shift(1) == 1) &
                            (df['hour_ny'].isin([3, 10, 14]))).astype(int)

    return df

### Trade

In [ ]:
##TRADE
def trade(df, initial_balance=100, risk_percentage=10):
    df = df.copy()
    df = df.reset_index(drop=False)  # Convert index to numeric
    balance = initial_balance  # Set initial balance in USD
    df['balance'] = initial_balance  # Initialize balance column with the initial balance

    for i in range(3, len(df)):
        # Buy Condition (Bullish FVG Retest)
        if df.loc[i, 'bullish_fvg_retest'] == 1:
            entry_price = df.loc[i, 'Low']
            stop_loss = df.loc[i - 3, 'Low']
            take_profit = entry_price + (2 * (entry_price - stop_loss))  # 2x risk reward
            price_risk = entry_price - stop_loss
            risk_percentage_btc = price_risk / entry_price  # Risk in terms of BTC price

            # Ensure risk percentage is within desired bounds
            if risk_percentage_btc > risk_percentage:
                continue  # Skip the trade if the risk is higher than the specified percentage

            # Log the trade signal
            df.loc[i, 'trade_signal'] = 1  # Buy Signal
            df.loc[i, 'entry_price'] = entry_price
            df.loc[i, 'stop_loss'] = stop_loss
            df.loc[i, 'take_profit'] = take_profit

            # Simulate trade exit based on conditions (SL, TP, or 240 rows)
            for j in range(i + 1, min(i + 240, len(df))):
                close_price = df.loc[j, 'Close']
                if close_price >= take_profit:  # Win
                    balance += balance * risk_percentage_btc
                    df.loc[i, 'W/L'] = 1
                    # Update balance from this row onward
                    df.loc[i:, 'balance'] = balance
                    print(f"Trade {i} Win: New Balance = {balance}")
                    break
                elif close_price <= stop_loss:  # Loss
                    balance -= balance * risk_percentage_btc
                    df.loc[i, 'W/L'] = -1
                    # Update balance from this row onward
                    df.loc[i:, 'balance'] = balance
                    print(f"Trade {i} Loss: New Balance = {balance}")
                    break
                if j == i + 239:  # If no TP or SL hit within 240 rows, close position
                    balance -= balance * risk_percentage_btc  # Deduct risk
                    df.loc[i, 'W/L'] = -1
                    # Update balance from this row onward
                    df.loc[i:, 'balance'] = balance
                    print(f"Trade {i} Timeout: New Balance = {balance}")

        # Sell Condition (Bearish FVG Retest)
        elif df.loc[i, 'bearish_fvg_retest'] == 1:
            entry_price = df.loc[i, 'High']
            stop_loss = df.loc[i - 3, 'High']
            take_profit = entry_price - (2 * (entry_price - stop_loss))  # 2x risk reward
            price_risk = stop_loss - entry_price
            risk_percentage_btc = price_risk / entry_price  # Risk in terms of BTC price

            # Ensure risk percentage is within desired bounds
            if risk_percentage_btc > risk_percentage:
                continue  # Skip the trade if the risk is higher than the specified percentage

            # Log the trade signal
            df.loc[i, 'trade_signal'] = -1  # Sell Signal
            df.loc[i, 'entry_price'] = entry_price
            df.loc[i, 'stop_loss'] = stop_loss
            df.loc[i, 'take_profit'] = take_profit

            # Simulate trade exit based on conditions (SL, TP, or 240 rows)
            for j in range(i + 1, min(i + 240, len(df))):
                close_price = df.loc[j, 'Close']
                if close_price <= take_profit:  # Win
                    balance += balance * risk_percentage_btc
                    df.loc[i, 'W/L'] = 1
                    # Update balance from this row onward
                    df.loc[i:, 'balance'] = balance
                    print(f"Trade {i} Win: New Balance = {balance}")
                    break
                elif close_price >= stop_loss:  # Loss
                    balance -= balance * risk_percentage_btc
                    df.loc[i, 'W/L'] = -1
                    # Update balance from this row onward
                    df.loc[i:, 'balance'] = balance
                    print(f"Trade {i} Loss: New Balance = {balance}")
                    break
                if j == i + 239:  # If no TP or SL hit within 240 rows, close position
                    balance -= balance * risk_percentage_btc  # Deduct risk
                    df.loc[i, 'W/L'] = -1
                    # Update balance from this row onward
                    df.loc[i:, 'balance'] = balance
                    print(f"Trade {i} Timeout: New Balance = {balance}")

    return df

## Live Trade Func

### Live_Detect_fvg

In [ ]:
##FVG
def live_detect_fvg(df):
    """
    FVG'leri tespit eder ve retest bekler.
    İşlem noktalarını belirleyip stop ve hedefleri hesaplar.
    """
    df = df.copy()

    # Bullish FVG: 1. mumun high'ı, 3. mumun low'undan küçük olmalı
    df['bullish_fvg'] = (df['High'].shift(3) < df['Low'].shift(1)).astype(int)


    # Bearish FVG: 1. mumun low'u, 3. mumun high'ından büyük olmalı
    df['bearish_fvg'] = (df['Low'].shift(3) > df['High'].shift(1)).astype(int)

    # Retest kontrolü: FVG bölgesine fiyat tekrar dokundu mu?
    df['bullish_fvg_retest'] = ((df['Low'] <= df['High'].shift(3)) &
                            (df['bullish_fvg'].shift(1) == 1)).astype(int)

    df['bearish_fvg_retest'] = ((df['High'] >= df['Low'].shift(2)) &
                                (df['bearish_fvg'].shift(1) == 1)).astype(int)

    return df

### live_trade

In [ ]:
def live_trade(df, initial_balance=100):
    df = df.copy()
    df = df.reset_index(drop=False)  # Ensure proper indexing
    balance = initial_balance  # Initial balance in USD
    trade_log = []  # List to store trade details

    for i in range(3, len(df)):
        trade_details = {
            'Time': None,
            'Entry': None,
            'SL': None,
            'TP': None,
            'Risk Amount': None,
            'Reward Amount': None,
            'RRR': None,
            'Close Time': None,
            'Duration': None,
            'Result': None,
            'Balance': balance  # Log the balance at the time of trade
        }

        if df.loc[i, 'bullish_fvg_retest'] == 1:
            entry_price = df.loc[i, 'Low']
            stop_loss = df.loc[i - 3, 'Low']
            take_profit = entry_price + 2 * (entry_price - stop_loss)  # 2x risk-reward
            risk_amount = (entry_price - stop_loss) / entry_price
            reward_amount = abs(risk_amount * 2)  # 2:1 RR
            entry_time = df.loc[i, 'Timestamp']
            trade_details.update({
                'Time' : entry_time,
                'Entry': entry_price,
                'SL': stop_loss,
                'TP': take_profit,
                'Risk Amount': abs(risk_amount),
                'Reward Amount': reward_amount,
                'RRR': 2
            })



        elif df.loc[i, 'bearish_fvg_retest'] == 1:
            entry_price = df.loc[i, 'High']
            stop_loss = df.loc[i - 3, 'High']
            take_profit = entry_price - 2 * (stop_loss - entry_price)
            risk_amount = (stop_loss - entry_price) /entry_price
            reward_amount = abs(risk_amount * 2)
            entry_time = df.loc[i, 'Timestamp']
            trade_details.update({
                'Time': entry_time,
                'Entry': entry_price,
                'SL': stop_loss,
                'TP': take_profit,
                'Risk Amount': abs(risk_amount),
                'Reward Amount': reward_amount,
                'RRR': 2
            })


        if trade_details['Entry'] is not None:
            trade_log.append(trade_details)

    return pd.DataFrame(trade_log)

                  Time         Entry            SL          TP  \
0 2025-03-25 15:03:46.632618  88016.171875  88004.164062  88040.1875   
1 2025-03-25 15:04:47.731744  88084.125000  87982.656250  88287.0625   

      Risk Amount  Reward Amount  RRR Close Time Duration Result  Balance  \
0    -0.000136      -0.000273    2       None     None   None      100  
1    -0.001152      -0.002304    2       None     None   None      100  

### check_pos

In [ ]:
def check_pos(df1, df2):
    df1 = df1.copy()
    df2 = df2.copy()
    df2 = df2.reset_index(drop=False)  # DataFrame'in indeksini düzenle

    for i in range(len(df1)):
        if pd.isna(df1.iloc[i]['Close Time']):  # Pozisyon açıksa kontrol et
            entry_time = df1.iloc[i]['Time']
            entry_price = df1.iloc[i]['Entry']
            stop_loss = df1.iloc[i]['SL']
            take_profit = df1.iloc[i]['TP']
            balance = df1.iloc[i]['Balance']  # Mevcut bakiye

            entry_time = entry_time.tz_localize(None) if entry_time.tzinfo else entry_time

            result = None
            close_time = None

            for j in range(len(df2)):
                row2_timestamp = df2.iloc[j]['Timestamp']
                row2_timestamp = row2_timestamp.tz_localize(None) if row2_timestamp.tzinfo else row2_timestamp

                if row2_timestamp > entry_time:  # İşleme girişten sonra kontrol et
                    current_price = df2.iloc[j]['Close']

                    if current_price >= take_profit:  # TP vurdu
                        result = 'Win'
                        balance += (df1.iloc[i]['Reward Amount'])*balance
                        close_time = row2_timestamp
                        break
                    elif current_price <= stop_loss:  # SL vurdu
                        result = 'Loss'
                        balance -= (df1.iloc[i]['Risk Amount'])*balance
                        close_time = row2_timestamp
                        break
                    elif (row2_timestamp - entry_time).seconds >= 14400:  # 4 saat limit
                        result = 'Timeout'
                        balance -= (df1.iloc[i]['Risk Amount'])*balance
                        close_time = row2_timestamp
                        break
                    else:
                        result = 'Open'


            if close_time != None:
              # İşlem güncellemesi
              df1.at[i, 'Close Time'] = close_time
              df1.at[i, 'Duration'] = (close_time - entry_time).seconds if close_time else None
              df1.at[i, 'Result'] = result
              df1.at[i, 'Balance'] = balance

    return df1  # Güncellenmiş df1 döndürülüyor


# Live Trade


In [ ]:
t_df = pd.DataFrame()
while True:
    # Download the data
    dfr = yf.download(tickers="BTC-USD", period="1d", interval="5m")
    # Rename the columns
    dfr.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    # Reorder the columns
    desired_order = ["Open", "Close", "High", "Low", "Volume"]
    dfr = dfr[desired_order]
    # Rename the index to Timestamp
    dfr.index.rename("Timestamp", inplace=True)
    # Convert the timezone from UTC to New York (Eastern Time) using tz_convert
    dfr.index = dfr.index.tz_convert('America/New_York')
    dfr = dfr[-5:]
    df = live_detect_fvg(dfr)
    df_a = live_trade(df)

    if not df_a.empty and t_df.empty:
      t_df = pd.concat([t_df, df_a], ignore_index=True)

    elif not df_a.empty and not t_df.empty:
      if not df_a['Time'].isin(t_df['Time']).any():  # df_a içindeki 'Time' değerleri t_df içinde var mı?
          t_df = pd.concat([t_df, df_a], ignore_index=True)
          print(f"\n t_df concat with df_A ============================\n{df_a} \n")
      else:
          print("\n No new unique entries in df_A, skipping concatenation.\n")

    if not t_df.empty:
      if t_df['Close Time'].isna().any():
        t_df = check_pos(t_df, df)
        print(f"\n t_df ============================\n{t_df }")

    time.sleep(301)



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed



 t_df ============================
                       Time         Entry            SL         TP  \
0 2025-03-26 17:05:00-04:00  87231.265625  87311.710938  87070.375   

   Risk Amount  Reward Amount  RRR Close Time Duration Result  Balance  
0     0.000922       0.001844    2       None     None   None      100  


[*********************100%***********************]  1 of 1 completed
<ipython-input-82-744907b4001d>:50: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '100.184441465852' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df1.at[i, 'Balance'] = balance



 t_df ============================
                       Time         Entry            SL         TP  \
0 2025-03-26 17:05:00-04:00  87231.265625  87311.710938  87070.375   

   Risk Amount  Reward Amount  RRR           Close Time Duration Result  \
0     0.000922       0.001844    2  2025-03-26 17:10:00      300    Win   

      Balance  
0  100.184441  


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


# Telegram

In [ ]:
!pip install python-telegram-bot --quiet
import telegram
from telegram import Bot
import asyncio
from telegram.error import RetryAfter

# Your Telegram bot token (replace with the token from BotFather)
TOKEN = '7853757542:AAHqR9gjgTOYmmV-W-n-QWvm63Ys1vJbKGQ'

# Your Telegram chat ID (you can find it from getUpdates)
CHAT_ID = '6079129575'

symbol = "BTC-USD"
bot = Bot(token=TOKEN)

async def send_message_with_retry(bot, chat_id, message):
    try:
        await bot.send_message(chat_id=chat_id, text=message)
        print("Sended")
    except RetryAfter as e:
        print(f"Flood limit exceeded. Retrying in {e.retry_after} seconds...")
        await asyncio.sleep(e.retry_after)  # Wait before retrying
        await send_message_with_retry(bot, chat_id, message)  # Retry after waiting


await send_message_with_retry(bot=bot,chat_id=CHAT_ID, message=t_df.iloc[-1].to_string())

Sended


#Bybit

In [ ]:
!pip install pybit --quiet
from pybit.unified_trading import HTTP
# Replace with your API keys
API_KEY = "CkvmeTWrwCTGyeP7KH"
API_SECRET = "jby5eGTKWJjJpSc1jOevYp51u8aYN1DBtQ4B"

# Initialize Bybit API
session = HTTP(
    api_key=API_KEY,
    api_secret=API_SECRET,
    testnet=True  # Set to False for live trading
)

# Check account balance
balance = session.get_wallet_balance(accountType="UNIFIED")
print(balance)